In [1]:
import pandas as pd
import numpy as np
from numpy import array, nan

from os.path import join
from eyefeatures.preprocessing.blinks_extraction import detect_blinks_pupil_missing, detect_blinks_pupil_vt, detect_blinks_eo

In [2]:
data = pd.read_csv(join('..', 'data', 'blinks', 'participant0003.tsv'), sep='\t', low_memory=False)
data = data[[
    'Participant name', 'Recording name',
    'Recording resolution height', 'Recording resolution width',
    'Eyetracker timestamp',
    'Gaze point X', 'Gaze point Y',
    'Pupil diameter right', 'Pupil diameter left',
    'Eye movement type'
]]
def str2num(v):
    try:
        if isinstance(v, str):
            return float(v.replace(',', '.'))
        if np.isnan(v):
            return np.nan
        return v
    except Exception as e:
        print(v)
        raise e

num_cols = [
    'Recording resolution height', 'Recording resolution width',
    'Eyetracker timestamp',
    'Gaze point X', 'Gaze point Y',
    'Pupil diameter right', 'Pupil diameter left',
]
for c in num_cols:
    print(c)
    data[c] = data[c].apply(str2num)

data = data.dropna(subset=['Eyetracker timestamp'])
data

Recording resolution height
Recording resolution width
Eyetracker timestamp
Gaze point X
Gaze point Y
Pupil diameter right
Pupil diameter left


,Participant name,Recording name,Recording resolution height,Recording resolution width,Eyetracker timestamp,Gaze point X,Gaze point Y,Pupil diameter right,Pupil diameter left,Eye movement type
1,Participant0001,Recording1,1080,1920,4.865500e+08,NaN,NaN,NaN,NaN,EyesNotFound
2,Participant0001,Recording1,1080,1920,4.865583e+08,NaN,NaN,NaN,NaN,EyesNotFound
3,Participant0001,Recording1,1080,1920,4.865667e+08,796.0,677.0,NaN,NaN,Saccade
4,Participant0001,Recording1,1080,1920,4.865750e+08,805.0,717.0,3.13,3.19,Saccade
5,Participant0001,Recording1,1080,1920,4.865833e+08,831.0,706.0,NaN,NaN,Saccade
...,...,...,...,...,...,...,...,...,...,...
179947,Participant0003,Recording16,1080,1920,1.658733e+09,934.0,504.0,NaN,NaN,Unclassified
179948,Participant0003,Recording16,1080,1920,1.658742e+09,953.0,517.0,NaN,NaN,Unclassified
179949,Participant0003,Recording16,1080,1920,1.658750e+09,770.0,558.0,NaN,2.11,Unclassified
179950,Participant0003,Recording16,1080,1920,1.658758e+09,748.0,551.0,NaN,NaN,Unclassified


In [3]:
data['Eye movement type'].value_counts()

Eye movement type
Fixation        76079
Saccade         51699
Unclassified    33884
EyesNotFound    13220
Name: count, dtype: int64

In [4]:
len(data)

174882

In [5]:
def show_results(ps, ts, ibs):
    print("is_blink | time   | size")
    for p, t, ib in zip(ps, ts, ibs):
        print(f"{ib}        | {t:.0f} | {p:.2f}")

In [6]:
test_pupil_sizes = array([1.2 , 1.12, 1.15, 1.3 , 1.21, 1.25, 0.9 ,  nan,  nan,  nan,  nan, 0.98,
   0.95, 1.2 , 1.33, 1.54, 1.3 , 1.3 , 1.25, 1.44])
test_timestamps = array([487383.208, 487391.543, 487399.885, 487408.216, 487416.551,
   487424.872, 487433.203, 487441.537, 487449.87 , 487458.203,
   487466.532, 487474.868, 487483.206, 487491.528, 487499.912,
   487508.225, 487516.531, 487524.865, 487533.192, 487540.733])

In [7]:
test_df, test_is_blink = detect_blinks_pupil_missing(test_pupil_sizes, test_timestamps, return_mask=True)

In [8]:
test_df

,onset,offset,duration
0,487433.203,487474.868,41.665


In [9]:
show_results(test_pupil_sizes, test_timestamps, test_is_blink)

is_blink | time   | size
0        | 487383 | 1.20
0        | 487392 | 1.12
0        | 487400 | 1.15
0        | 487408 | 1.30
0        | 487417 | 1.21
0        | 487425 | 1.25
1        | 487433 | 0.90
1        | 487442 | nan
1        | 487450 | nan
1        | 487458 | nan
1        | 487467 | nan
1        | 487475 | 0.98
0        | 487483 | 0.95
0        | 487492 | 1.20
0        | 487500 | 1.33
0        | 487508 | 1.54
0        | 487517 | 1.30
0        | 487525 | 1.30
0        | 487533 | 1.25
0        | 487541 | 1.44


In [10]:
detect_blinks_pupil_missing(data['Pupil diameter right'].values, data['Eyetracker timestamp'].values / 1e3)

,onset,offset,duration
0,486549.983,1658766.763,1172216.78


In [11]:
test_df, test_is_blink = detect_blinks_pupil_vt(test_pupil_sizes, test_timestamps, Fs=120, return_mask=True)

In [12]:
test_df

,onset,offset,duration
0,487441.537,487466.532,24.995


In [13]:
show_results(test_pupil_sizes, test_timestamps, test_is_blink)

is_blink | time   | size
0        | 487383 | 1.20
0        | 487392 | 1.12
0        | 487400 | 1.15
0        | 487408 | 1.30
0        | 487417 | 1.21
0        | 487425 | 1.25
0        | 487433 | 0.90
1        | 487442 | nan
1        | 487450 | nan
1        | 487458 | nan
1        | 487467 | nan
0        | 487475 | 0.98
0        | 487483 | 0.95
0        | 487492 | 1.20
0        | 487500 | 1.33
0        | 487508 | 1.54
0        | 487517 | 1.30
0        | 487525 | 1.30
0        | 487533 | 1.25
0        | 487541 | 1.44


In [14]:
detect_blinks_pupil_vt(data['Pupil diameter right'].values, data['Eyetracker timestamp'].values / 1e3, Fs=120)

,onset,offset,duration
0,486549.983,486766.621,216.638
1,490107.964,490216.263,108.299
2,490457.856,490641.172,183.316
3,492982.571,493115.891,133.320
4,495882.272,496040.589,158.317
...,...,...,...
1291,1657083.616,1657391.915,308.299
1292,1657758.600,1657966.858,208.258
1293,1658083.504,1658166.837,83.333
1294,1658233.507,1658266.831,33.324


In [15]:
test_df, test_eo_vel = detect_blinks_eo(test_pupil_sizes, test_timestamps, Fs=120, return_eo_vel=True)

In [16]:
test_eo_vel

array([ -4.8,  -3. ,  10.8,   3.6,  -3. , -18.6,   nan,   nan,   nan,
         nan,   nan,   nan,  13.2,  22.8,  20.4,  -1.8, -14.4,  -3. ,
         8.4,  11.4])

In [17]:
test_df

,onset,offset,duration,time_peak,openness_at_onset,openness_at_offset,openness_at_peak,time_peak_opening_velocity,time_peak_closing_velocity,peak_opening_velocity,peak_closing_velocity,opening_amplitude,closing_amplitude


In [18]:
detect_blinks_eo(data['Pupil diameter right'].values, data['Eyetracker timestamp'].values / 1e3, Fs=120)

,onset,offset,duration,time_peak,openness_at_onset,openness_at_offset,openness_at_peak,time_peak_opening_velocity,time_peak_closing_velocity,peak_opening_velocity,peak_closing_velocity,opening_amplitude,closing_amplitude
0,505247.855,505314.508,66.653,505297.873,3.340000,3.016667,2.57,505306.178,505281.188,26.8,-16.8,0.446667,0.770000
1,978788.560,979605.138,816.578,979330.127,3.473333,3.140000,2.75,979588.481,978813.549,14.8,-26.0,0.390000,0.723333
2,1051780.328,1051963.643,183.315,1051872.041,3.056667,3.410000,2.52,1051946.975,1051805.380,20.2,-14.4,0.890000,0.536667
3,1118753.318,1118820.024,66.706,1118803.334,2.960000,2.493333,2.16,1118811.698,1118770.014,20.0,-24.8,0.333333,0.800000
4,1215680.773,1215747.364,66.591,1215722.426,3.276667,2.890000,2.57,1215730.698,1215705.702,12.8,-17.6,0.320000,0.706667
5,1249326.936,1249393.649,66.713,1249368.600,3.076667,3.210000,2.66,1249376.962,1249351.964,22.0,-12.4,0.550000,0.416667
6,1298196.466,1298263.115,66.649,1298238.154,3.180000,3.060000,2.60,1298246.550,1298221.452,18.4,-16.8,0.460000,0.580000
7,1334467.391,1334550.723,83.332,1334509.127,2.993333,2.886667,2.56,1334542.498,1334492.390,14.8,-12.8,0.326667,0.433333
8,1420452.598,1420510.918,58.320,1420494.262,3.196667,3.093333,2.74,1420502.593,1420477.594,21.2,-14.0,0.353333,0.456667
9,1433926.084,1434309.376,383.292,1434192.721,3.366667,3.070000,2.66,1434301.112,1433951.082,15.6,-15.6,0.410000,0.706667
